# Customer Segementation

In [60]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import statsmodels.api as sm
import time
from datetime import datetime,timedelta
import warnings
warnings.filterwarnings("ignore") 
import scipy.stats as stats

In [61]:
# df = pd.read_csv("../updated_pos_no2.csv")
df = pd.read_csv('Transformed_data.csv')

In [62]:
df['Date'].max()
df['Date'] = pd.to_datetime(df['Date'])
snapshot_day = df['Date'].max() + timedelta(days=1)
days_ago_series = (snapshot_day-df['Date']).astype('timedelta64[D]').astype(int)

In [63]:
RFM = df.groupby('Customer ID').agg(Recency=('Date',lambda x :(snapshot_day - x.max())))
RFM['Frequency']= df.groupby('Customer ID').agg(Frequency=('Bill No','count'))
RFM['Monetary'] = df.groupby('Customer ID').agg(Monetary=('Amount','sum'))

In [64]:
recency_labels = range(4,0,-1) #lowest value is given the highest ranking coz it tells that person has bought recently 

RFM['R_score'] = pd.qcut(RFM['Recency'], q=4,labels=recency_labels)

Frequency_labels = range(1,5)

RFM['F_score'] = pd.qcut(RFM['Frequency'], q=4,labels=Frequency_labels)

Monetary_labels = range(1,5)

RFM['M_score'] = pd.qcut(RFM['Monetary'], q=4,labels=Monetary_labels)

RFM['rfm_total_score'] = RFM['R_score'].astype(int)+RFM['F_score'].astype(int)+RFM['M_score'].astype(int)

RFM_Segments = (RFM['R_score'].astype(str) + '.'+ RFM['F_score'].astype(str)+'.' + RFM['M_score'].astype(str))

RFM['Segments'] = RFM_Segments

In [65]:
def group_function(df):
    if df['rfm_total_score'] > 9:
        return "Elite Customers"
    elif df['rfm_total_score'] > 5 and df['F_score'] == 4:
        return 'Frequent Customers'
    elif df['rfm_total_score'] <= 5 and df['F_score'] == 1:
        return 'Rare Customer'
    elif df['rfm_total_score'] <= 5:
        return "Cost-conscious Customers"
    else:
        return 'Moderate Value Customers'

In [68]:
RFM['Group']= RFM.apply(group_function,axis=1)
if(len(RFM.columns) == 9):
    RFM.reset_index(inplace=True)
customer_segments_df = RFM[['Customer ID', 'Group']]
customer_segments_df = customer_segments_df.reset_index(drop=True)
customer_segments_df
customer_segments_df.to_csv('customer_segments.csv')